# Python Built-in Functions and Runtime Library

This notebook provides a comprehensive guide to Python's built-in functions and runtime library. You'll learn how to interact with the operating system, manage files and directories, execute external programs, handle logging, and work with various data formats.

## Table of Contents

1. **System Functions**
   - Environment Variables
   - System Path Management
   - Working Directory Operations
   - File and Directory Listing
   - Directory Manipulation
   - File Information Queries
   - Path Management with `os.path` and `pathlib`
   - Temporary Files

2. **Process Management**
   - Executing External Programs
   - Subprocess Module
   - Process Communication and Control

3. **Logging**
   - Basic Logging
   - Custom Loggers
   - Log Formatting

4. **Data Serialization**
   - JSON Format
   - Pickle Format
   - Data Persistence Best Practices

5. **Data Archiving and Compression**
   - TAR Files
   - ZIP Files
   - Compression Techniques

6. **Special File Formats**
   - Configuration Files (INI)
   - CSV Files
   - Advanced File Processing

## Learning Objectives

By the end of this notebook, you will be able to:
- Interact with the operating system using Python
- Manage files and directories programmatically
- Execute and control external processes
- Implement structured logging in applications
- Serialize and deserialize data in various formats
- Work with compressed archives and special file formats

---

# System functions

## Environment variables

Environment variables are variables defined in the operating environment. Users can modify these to control behavior of applications and software. In Python the dirctionary os.environ contains these variables.

In [ ]:
import os

# Safe way to access environment variables with default values
print("PATH:", os.environ.get("PATH", "PATH not found"))

# Display only first few environment variables to avoid cluttering output
print("\nFirst 5 environment variables:")
for i, (variable, value) in enumerate(os.environ.items()):
    if i < 5:
        print(f"{variable} = {value[:50]}{'...' if len(value) > 50 else ''}")
    else:
        break

print(f"\nTotal environment variables: {len(os.environ)}")

# Setting and getting custom environment variables
os.environ["PYTHON_COURSE_DEMO"] = "Hello from Python!"
print(f"\nCustom variable: {os.environ.get('PYTHON_COURSE_DEMO')}")

# Best practice: Check if variable exists before accessing
if "HOME" in os.environ:
    print(f"Home directory: {os.environ['HOME']}")
elif "USERPROFILE" in os.environ:  # Windows equivalent
    print(f"User profile: {os.environ['USERPROFILE']}")
else:
    print("Home directory not found in environment variables")

## Accessing the system path

The system path controls how the operating system searches for executables in the file system. Python has a neutral way of querying these from the system.

In [ ]:
import os

exe_path_list = os.get_exec_path()

for path in exe_path_list:
    print(path)

## Changing and querying the working directory

The working directory is the directory where your application is started. Python has functions for changing and querying the working directory.

In [ ]:
import os

cwd = os.getcwd()
print(cwd)

os.chdir("..")
print(os.getcwd())
os.chdir(cwd)
print(os.getcwd())

## Listing files in a directory

For many applications it can be important to query what files and directories are available. The os.listdir() function implements this in Python.

In [ ]:
# Create a test file using Python (cross-platform)
with open("testfile", "w") as f:
    f.write("This is a test file created by Python")

In [ ]:
import os

for item in os.listdir():
    print(item)

for item in os.listdir():
    if os.path.isdir(item):
        print("Katalog:", item)
    if os.path.isfile(item):
        print("Fil    :", item)

## Directory manipulation

Python also has a lot of functions for creating and manipulating directories.

In [ ]:
import os
from pathlib import Path

# Store current directory for cleanup
cwd = os.getcwd()

try:
    # Create directory if it doesn't exist
    test_dir = Path("demo_directory")
    test_dir.mkdir(exist_ok=True)
    
    # Change to the new directory
    os.chdir(test_dir)
    print(f"Changed to: {os.getcwd()}")
    
    # Create subdirectory and file
    subdir = Path("testdir")
    subdir.mkdir(exist_ok=True)
    
    test_file = Path("testfile.txt")
    test_file.write_text("This is test content")
    
    print("Directory contents after creation:")
    for item in os.listdir():
        item_path = Path(item)
        if item_path.is_dir():
            print(f"  📁 Directory: {item}")
        else:
            print(f"  📄 File: {item} ({item_path.stat().st_size} bytes)")
    
    # Rename the file
    new_name = Path("renamed_testfile.txt")
    test_file.rename(new_name)
    
    print("\nDirectory contents after rename:")
    for item in os.listdir():
        print(f"  {item}")
    
    # Cleanup
    new_name.unlink()  # Remove file
    subdir.rmdir()     # Remove empty directory
    
    print("\nDirectory contents after cleanup:")
    print(f"  Items remaining: {len(os.listdir())}")
    
finally:
    # Always return to original directory
    os.chdir(cwd)
    # Remove the demo directory if empty
    try:
        test_dir.rmdir()
        print("Demo directory removed")
    except OSError as e:
        print(f"Could not remove demo directory: {e}")

## Listing and querying file information

The os.scandir() function can be used to query more detailed information on the a file or directory.

In [ ]:
import os

with os.scandir() as items:
    for entry in items:
        print("------------------------")
        print("name", entry.name)
        print("path", entry.path)
        print("is_dir", entry.is_dir())
        print("is_file", entry.is_file())

## Walking directories with os.walk()

The os.walk() function enables you to traverse directories as an iteration using the for-statement.

In [ ]:
import os

for root, dirs, files in os.walk("."):
    print("--->")
    print(root)
    print(dirs)
    print(files)
    print("<---")

## Querying file information

On Unix-based platform the os.stat() function can return detailed information on files and directories.

In [ ]:
import os

with open("testfile", "w") as f:
    f.write("testfile")

os.mkdir("testdir2")

statinfo_file = os.stat("testfile")
statinfo_dir = os.stat("testdir2")

print(statinfo_file)
print(statinfo_dir)
print(statinfo_file.st_size)

## Path information

The os.path module contains even more functions for querying path and file information.

In [ ]:
import os

print(os.path.abspath('.'))
print(os.path.basename('/home/user/test.txt'))
print(os.path.dirname('/home/user/test.txt'))

with open("testfile_path", "w") as f:
    f.write("testfile")

if os.path.exists('/home/user/test.txt'):
    print('test.txt is valid')
else:
    print('test.txt is not valid')

print(os.path.expanduser('~'))

print(os.path.getatime('testfile_path'))
print(os.path.getmtime('testfile_path'))
print(os.path.getctime('testfile_path'))

print(os.path.getsize('testfile_path'))

if os.path.isabs('testfile_path'):
    print('Absolute path')
else:
    print('No absolute path')

if os.path.isabs("C:/Users/jonas/Development/python_book/examples/rtl/ospath1.py"):
    print('Absolute path')
else:
    print('No absolute path')

if os.path.isfile('testfile_path'):
    print('ospath1.py is a file')
else:
    print('ospath1.py is not a file')

if os.path.isdir('testfile_path'):
    print('ospath1.py is a directory')
else:
    print('ospath1.py is not a directory')

dir_name = 'c:\\Users\\jonas'
file_name = 'test.txt'

file_path = os.path.join(dir_name, file_name)

print(file_path)
print(os.path.split(file_path))
dir_name, file_name = os.path.split(file_path)
print(dir_name)
print(file_name)
print(os.path.splitdrive(file_path))
print(os.path.splitext(file_path))

## Path management using pathlib

pathlib is a module implementing OO-based file and path management.

In [ ]:
import pathlib as pl

p = pl.Path('/contents')
p = p / "testfile"
print(p)

print(p.exists())

q = p.resolve()
print(q)


print(q.parts)
print(q.drive)

r = pl.Path.cwd()
print(r)

print(r.exists())
print(r.is_dir())
print(r.is_file())


s = pl.Path.home()
print(s)

## Iterating with pathlib

In [ ]:
import pathlib as pl

p = pl.Path(".")

for x in p.iterdir():
    if x.is_dir():
        print(x,'- katalog')
    else:
        print(x,'- fil')

## Changing current path with pathlib

In [ ]:
import os
import pathlib as pl

p = pl.Path('..')

os.chdir(p)

q = pl.Path.cwd()
print(q)

In [ ]:
import os
import pathlib as pl

new_path = pl.Path('..')
old_path = pl.Path.cwd()

os.chdir(new_path)

print(pl.Path.cwd())

os.chdir(old_path)

print(pl.Path.cwd())

## Temporary files

In many applications you need to create temporary files. In Python there are functions for securely creating temporary files.

In [ ]:
import os
import tempfile
from pathlib import Path

# Method 1: Using mkstemp (lower-level, more control)
print("=== Method 1: Using mkstemp ===")
temp_fd, temp_path = tempfile.mkstemp(suffix='.txt', prefix='python_demo_')

try:
    print(f'Temporary file created: {temp_path}')
    print(f'File exists: {os.path.isfile(temp_path)}')
    print(f'File permissions: {oct(os.stat(temp_path).st_mode)[-3:]}')
    
    # Write to the temporary file using the file descriptor
    with os.fdopen(temp_fd, 'w+t') as temp_file:
        temp_file.write('This is written to the temporary file\n')
        temp_file.write('Line 2 of content\n')
        
        # Read back the content
        temp_file.seek(0)
        content = temp_file.read()
        print(f'File content:\n{content}')
        
finally:
    # Important: Always clean up temporary files
    if os.path.exists(temp_path):
        os.remove(temp_path)
        print(f'Temporary file cleaned up: {not os.path.exists(temp_path)}')

print("\n=== Method 2: Using TemporaryFile (recommended) ===")
# Method 2: Using TemporaryFile (automatically cleaned up)
with tempfile.TemporaryFile(mode='w+t', suffix='.txt') as temp_file:
    temp_file.write('This content will be automatically cleaned up\n')
    temp_file.write('No manual cleanup required!\n')
    
    # Read back
    temp_file.seek(0)
    content = temp_file.read()
    print(f'Content: {content.strip()}')
    
    # File is automatically deleted when exiting the with block

print("\n=== Method 3: Using NamedTemporaryFile ===")
# Method 3: NamedTemporaryFile (has a name but still auto-cleaned)
with tempfile.NamedTemporaryFile(mode='w+t', suffix='.txt', delete=True) as temp_file:
    print(f'Named temporary file: {temp_file.name}')
    temp_file.write('Named temporary file content\n')
    temp_file.flush()  # Ensure content is written
    
    # You can access the file by name while it's open
    print(f'File size: {Path(temp_file.name).stat().st_size} bytes')

print("\n=== Temporary Directory ===")
# Bonus: Temporary directories
with tempfile.TemporaryDirectory(prefix='python_demo_') as temp_dir:
    print(f'Temporary directory: {temp_dir}')
    
    # Create files in the temporary directory
    temp_file_path = Path(temp_dir) / 'example.txt'
    temp_file_path.write_text('File in temporary directory')
    
    print(f'Files in temp dir: {list(Path(temp_dir).iterdir())}')
    # Directory and all contents automatically cleaned up

In [ ]:
import os, tempfile

with tempfile.TemporaryFile() as temp_file:
    print('Temporär fil', temp_file.name, 'skapad.')
    print(os.path.isfile(temp_file.name))
    temp_file.write(b'this is written to the temp file')
    temp_file.seek(0)

print(os.path.isfile(temp_file.name))

In [ ]:
result = os.system('ls -la')

# Process management

## Starting program using the subprocess module

In many cases there is a need for a Python script to execute other applications and scripts. Python implements a multitude of functions for executing processes. Most of them are implemented in the subprocess module.

In [ ]:
import subprocess
import sys
import platform

# Cross-platform directory listing command
if platform.system() == "Windows":
    cmd = ['dir']
    shell_needed = True
else:
    cmd = ['ls', '-la']
    shell_needed = False

print(f"Running on: {platform.system()}")
print(f"Command: {' '.join(cmd) if not shell_needed else cmd[0]}")

try:
    # Using subprocess.run() - the recommended modern approach
    result = subprocess.run(
        cmd, 
        shell=shell_needed,
        capture_output=True,  # Capture both stdout and stderr
        text=True,           # Return strings instead of bytes
        timeout=10           # Prevent hanging
    )
    
    print(f"Return code: {result.returncode}")
    
    if result.returncode == 0:
        print("✅ Process completed successfully")
        print(f"Output:\n{result.stdout}")
    else:
        print("❌ Process failed")
        print(f"Error output:\n{result.stderr}")
        
except subprocess.TimeoutExpired:
    print("❌ Process timed out")
except subprocess.SubprocessError as e:
    print(f"❌ Subprocess error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

# Demonstrate with Python command (cross-platform)
print("\n" + "="*50)
print("Running Python version check:")

try:
    result = subprocess.run(
        [sys.executable, '--version'],
        capture_output=True,
        text=True,
        timeout=5
    )
    
    if result.returncode == 0:
        print(f"✅ {result.stdout.strip()}")
    else:
        print(f"❌ Failed: {result.stderr}")
        
except Exception as e:
    print(f"❌ Error checking Python version: {e}")

In [ ]:
import subprocess

result = subprocess.run('ls -la', shell=True, stdout=subprocess.PIPE, universal_newlines=True)

if result.returncode == 0:
    print('Processen returnerade 0')
    print('Utdata:')
    print(result.stdout)
else:
    print('Processen returnerade felkoden = ', result.returncode)

In [ ]:
import subprocess

p = subprocess.Popen(['ls', '-la'])

# Other processing here...

p.wait()

if p.returncode == 0:
    print('Processen returnerade 0')
else:
    print('Processen returnerade felkoden = ', p.returncode)

In [ ]:
import subprocess, time

p = subprocess.Popen(['sleep', '5'])

while p.poll() is None:
    print('Väntar...')
    time.sleep(1)

if p.returncode == 0:
    print('Processen returnerade 0')
else:
    print('Processen returnerade felkoden = ', p.returncode)

In [ ]:
import subprocess, time

p = subprocess.Popen('ls -la; sleep 4', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

while p.poll() is None:
    print('Väntar...')
    time.sleep(1)

stdout, stderr = p.communicate()

if p.returncode == 0:
    print('Processen returnerade 0')
    print('standard output:')
    print(stdout)
    print('standard error:')
    print(stderr)
else:
    print('Processen returnerade felkoden = ', p.returncode)

In [ ]:
import subprocess

with subprocess.Popen('ls -la', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True) as p:
    stdout, stderr = p.communicate()

    if p.returncode == 0:
        print('Processen returnerade 0')
        print('standard output:')
        print(stdout)
        print('standard error:')
        print(stderr)
    else:
        print('Processen returnerade felkoden = ', p.returncode)

In [ ]:
import subprocess


def execute_with_output(cmd):

    with subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, universal_newlines=True) as p:
        stdout, _ = p.communicate()

        if p.returncode == 0:
            return stdout
        else:
            return None


if __name__ == "__main__":

    output = execute_with_output('ls')

    if output is not None:

        lines = output.split("\n")

        for line in lines:
            print('>' + line)
    else:
        print('Ingen utdata returnerades.')

# Logging

For larger applications there is often a need to create log entries in a more structured way. The logging module in Python implements this functionality.

In [ ]:
import logging
import sys
from datetime import datetime

# Configure logging with a better format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def simulate_application_workflow():
    """Simulate a real application with different logging scenarios"""
    logger = logging.getLogger('MyApplication')
    
    logger.info("🚀 Application starting...")
    
    # Simulate processing some data
    try:
        logger.debug("Processing user data...")
        
        # Simulate different scenarios
        users = ['alice', 'bob', 'charlie']
        
        for user in users:
            logger.info(f"Processing user: {user}")
            
            if user == 'charlie':
                logger.warning(f"⚠️  User {user} has incomplete profile")
            
        logger.info("✅ All users processed successfully")
        
    except Exception as e:
        logger.error(f"❌ Error processing users: {e}")
        logger.exception("Full traceback:")  # This includes the stack trace
    
    logger.info("🏁 Application workflow completed")

print("=== Logging with different levels ===")

# Test different logging levels
print("\n1. INFO level and above:")
logging.getLogger().setLevel(logging.INFO)
simulate_application_workflow()

print("\n2. WARNING level and above:")
logging.getLogger().setLevel(logging.WARNING)
simulate_application_workflow()

print("\n3. ERROR level only:")
logging.getLogger().setLevel(logging.ERROR)
simulate_application_workflow()

# Reset to INFO for subsequent examples
logging.getLogger().setLevel(logging.INFO)

In [ ]:
import logging

def do_some_logging2(logger):
    logger.debug("Detta är ett testmeddelande...")
    logger.info("Detta är ett informationsmeddelande...")
    logger.warning("Detta är en varning...")
    logger.error("Detta är ett felmeddelande...")
    logger.critical("Detta är ett kritiskt fel...")
    logger.critical("-----------------------------")


mylog = logging.getLogger("mylog")
mylog.setLevel(logging.DEBUG)

do_some_logging2(mylog)

print("Loglevel ERROR")

mylog.setLevel(logging.ERROR)

do_some_logging2(mylog)

In [ ]:
import logging

def do_some_logging(logger):
    logger.debug("Detta är ett testmeddelande...")
    logger.info("Detta är ett informationsmeddelande...")
    logger.warning("Detta är en varning...")
    logger.error("Detta är ett felmeddelande...")
    logger.critical("Detta är ett kritiskt fel...")

logging.basicConfig(level=logging.DEBUG, format='%(asctime)-15s %(name)-10s %(levelname)-8s %(message)s')

mylog = logging.getLogger("mylog")
mylog.setLevel(logging.DEBUG)

do_some_logging(mylog)

# Data serialization and deserialization

## Pickle files

In [ ]:
import pickle, zlib

my_data = {"a number": 42, "a list": list(range(1000)), "a dict": {'a': 1, 'b': 2}}

my_data_dump = pickle.dumps(my_data)
print(len(my_data_dump))

my_data_compressed = zlib.compress(my_data_dump)
print(len(my_data_compressed))

my_data_uncompressed = zlib.decompress(my_data_compressed)

my_data_copy = pickle.loads(my_data_uncompressed)
print(my_data_copy)

## Storing data structures using the standard pickle-format

The pickle-module is the standard way of storing Python data types in Python. It is used in the same way as the json-module.

In [ ]:
import pickle
import json
import time
from pathlib import Path

# Create test data of different types
test_data = {
    "integers": list(range(1000)),
    "strings": [f"string_{i}" for i in range(100)],
    "nested_dict": {
        "level1": {
            "level2": {
                "data": [1, 2, 3, 4, 5] * 50
            }
        }
    },
    "mixed_types": [1, "string", [1, 2, 3], {"key": "value"}]
}

print("=== Pickle vs JSON Comparison ===")
print(f"Data complexity: {len(str(test_data))} characters when converted to string")

# Test Pickle
print("\n1. PICKLE FORMAT:")
start_time = time.time()

# Save with pickle
with open("test_data.pkl", "wb") as f:
    pickle.dump(test_data, f)

pickle_save_time = time.time() - start_time
pickle_size = Path("test_data.pkl").stat().st_size

# Load with pickle
start_time = time.time()
with open("test_data.pkl", "rb") as f:
    pickle_loaded = pickle.load(f)
pickle_load_time = time.time() - start_time

print(f"  ✅ Save time: {pickle_save_time:.4f} seconds")
print(f"  ✅ Load time: {pickle_load_time:.4f} seconds")
print(f"  📦 File size: {pickle_size} bytes")
print(f"  🔍 Data integrity: {'✅ OK' if pickle_loaded == test_data else '❌ FAILED'}")

# Test JSON (only for JSON-compatible data)
json_compatible_data = {
    "integers": list(range(1000)),
    "strings": [f"string_{i}" for i in range(100)],
    "nested_dict": {
        "level1": {
            "level2": {
                "data": [1, 2, 3, 4, 5] * 50
            }
        }
    }
}

print("\n2. JSON FORMAT:")
start_time = time.time()

# Save with JSON
with open("test_data.json", "w") as f:
    json.dump(json_compatible_data, f)

json_save_time = time.time() - start_time
json_size = Path("test_data.json").stat().st_size

# Load with JSON
start_time = time.time()
with open("test_data.json", "r") as f:
    json_loaded = json.load(f)
json_load_time = time.time() - start_time

print(f"  ✅ Save time: {json_save_time:.4f} seconds")
print(f"  ✅ Load time: {json_load_time:.4f} seconds")
print(f"  📦 File size: {json_size} bytes")
print(f"  🔍 Data integrity: {'✅ OK' if json_loaded == json_compatible_data else '❌ FAILED'}")

print("\n=== SUMMARY ===")
print(f"🏎️  Speed - Save: {'Pickle' if pickle_save_time < json_save_time else 'JSON'} is faster")
print(f"🏎️  Speed - Load: {'Pickle' if pickle_load_time < json_load_time else 'JSON'} is faster")
print(f"💾 Size: {'Pickle' if pickle_size < json_size else 'JSON'} produces smaller files")
print(f"📖 Human readable: JSON ✅ | Pickle ❌")
print(f"🔧 Python-specific types: Pickle ✅ | JSON ❌")
print(f"🌐 Cross-language compatibility: JSON ✅ | Pickle ❌")

print("\n=== BEST PRACTICES ===")
print("🎯 Use PICKLE when:")
print("   • Working exclusively with Python")
print("   • Need to preserve exact Python object types")
print("   • Performance is critical")
print("   • Working with complex Python objects (classes, functions, etc.)")

print("\n🎯 Use JSON when:")
print("   • Need human-readable format")
print("   • Sharing data with other languages/systems")
print("   • Working with web APIs")
print("   • Data is relatively simple (dicts, lists, strings, numbers)")

# Cleanup
Path("test_data.pkl").unlink(missing_ok=True)
Path("test_data.json").unlink(missing_ok=True)

## JSON files

In [ ]:
import json

my_data = {"a number": 42, "a list": list(range(1000)), "a dict": {'a': 1, 'b': 2}}

my_data_dump = json.dumps(my_data)
print(len(my_data_dump))

my_data_copy = json.loads(my_data_dump)
print(my_data_copy)

with open("mydata.json", "w") as json_file:
    json.dump(my_data, json_file)

with open("mydata.json", "r") as json_file:
    my_data_copy = json.load(json_file)

In [ ]:
!ls
!cat mydata.json

## Storing variables and data structures (JSON)

Python has built in functions for writing data types to disk. If readability is important the Javascript Object Notation or JSON can be used as a storage format. In Python this functionality is found in the **json**-module.

In [ ]:
import json

my_data = {"a number": 42, "a list": [1, 2, 3, 4], "a dict": {'a': 1, 'b': 2}}

with open("mydata.json", "w") as my_file:
    json.dump(my_data, my_file)

with open("mydata.json", "r") as my_file:
    my_data_copy = json.load(my_file)

print(my_data_copy)
!cat mydata.json

In [ ]:
import json

my_data = {"a number": 42, "a list": [1, 2, 3, 4], "a dict": {'a': 1, 'b': 2}}

json_string = json.dumps(my_data)

print(json_string)

my_data_copy = json.loads(json_string)

print(my_data_copy)

In [ ]:
import json

my_data = {"a number": 42, "a list": [1, 2, 3, 4], "a dict": {'a': 1, 'b': 2}}

print(json.dumps(my_data, sort_keys=True, indent=4))

In [ ]:
import pickle

my_data = {"a number": 42, "a list": [1, 2, 3, 4], "a dict": {'a': 1, 'b': 2}}

with open("my_data_text.pkl", "wb") as my_file:
    pickle.dump(my_data, my_file, protocol=0)

with open("my_data_text.pkl", "rb") as my_file:
    my_data_copy = pickle.load(my_file)

print(my_data_copy)

In [ ]:
import pickle

my_data = {"a number": 42, "a list": [1, 2, 3, 4], "a dict": {'a': 1, 'b': 2}}

my_data_dump = pickle.dumps(my_data)

print(my_data_dump)

my_data_copy = pickle.loads(my_data_dump)

print(my_data_copy)

# Data archiving and compression

## Tarfiles

In [ ]:
!wget https://fossbytes.com/wp-content/uploads/2016/10/commodore64.jpg -O bild1.jpg

In [ ]:
!wget https://ichef.bbci.co.uk/news/640/media/images/68628000/jpg/_68628283_apple-1.jpg -O bild2.jpg

In [ ]:
import tarfile as tf

with tf.TarFile("myarchive.tar.gz", "w") as mytar:
    mytar.add("bild1.jpg")
    mytar.add("bild2.jpg")

with tf.TarFile("myarchive.tar.gz", "r") as mytar:
    print(mytar.getnames())
    print(mytar.getmembers())
    mytar.extract("bild1.jpg", "mytar")
    mytar.extractall("mytar_all")
    mytar.list(verbose=True)

In [ ]:
!ls

## Zip-files

In [ ]:
import zipfile as zf
import matplotlib.pyplot as plt

with zf.ZipFile("myarchive.zip", "w") as myzip:
    myzip.write("bild1.jpg")
    myzip.write("bild2.jpg")

with zf.ZipFile("myarchive.zip", "r") as myzip:
    print(myzip.namelist())
    print(myzip.getinfo("bild1.jpg"))
    myzip.extract("bild2.jpg", "myzip")
    myzip.extractall("myzip_all")
    myzip.printdir()
    #with myzip.open("bild1.jpg") as myfile:
    #    image1 = plt.imread(myfile)
    with myzip.open("bild2.jpg") as myfile:
        image2 = plt.imread(myfile)
    #plt.imshow(image1)
    plt.imshow(image2)
    plt.show()

# Special file formats

## Configuration files

In [ ]:
config_file = """[DEFAULT]
ServerAliveInterval = 45
Compression = yes
CompressionLevel = 9
ForwardX11 = yes

[bitbucket.org]
User = hg

[topsecret.server.com]
Port = 50022
ForwardX11 = no"""


with open("config1.ini", "w") as f:
    f.write(config_file)

import configparser

config = configparser.ConfigParser()
config.read("config1.ini")

sections = config.sections()
print(sections)

print(config["bitbucket.org"]["user"])

for section in config.sections():
    print("section =", section)
    keys = config[section].keys()
    for key in keys:
        print(key, "=", config[section][key])

config["bitbucket.org"]["user"] = "jonas"
print(config["bitbucket.org"]["user"])

with open("config2.ini", "w") as config_file:
    config.write(config_file)

In [ ]:

import configparser

config = configparser.ConfigParser()

config["DEFAULT"] = {
        "Rating":"No rating",
        "Length":"No length"
        }

config["Dr Who"] = {"Rating":"9/9"}
config["Firefly"] = {"Length":"Too long"}

with open("config3.ini", "w") as config_file:
    config.write(config_file)


In [ ]:
!cat config3.ini

## Comma separated files - CSV

In [ ]:
!rm example1.csv
!wget https://raw.githubusercontent.com/jonaslindemann/guide_to_python/master/chapters/kapitel4/notebook/example1.csv

In [ ]:
import csv

with open('example1.csv', 'r') as csv_file:
    csv_data = csv.reader(csv_file, delimiter=',')
    for row in csv_data:
        print(row)

with open('example2.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    csv_writer.writerow(['Beteckning', 'Antal'])
    csv_writer.writerow(['Gurka', '2'])
    csv_writer.writerow(['Tomat', '4'])

l = [["Beteckning", "Antal"],["Gurka", "2"], ["Tomat", "4"]]

with open('example3.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    csv_writer.writerows(l)

In [ ]:
!cat example1.csv

# 🎯 Practical Exercises and Challenges

Now that you've learned about Python's built-in functions and runtime library, here are some practical exercises to reinforce your understanding:

## Exercise 1: System Information Tool 🖥️

Create a Python script that gathers and displays system information:

**Requirements:**
- Display current working directory
- Show environment variables (HOME/USERPROFILE, PATH, PYTHONPATH)
- List files in current directory with their sizes
- Show Python version and executable path
- Display disk usage for current directory

**Bonus:** Save the information to a JSON file with timestamp.

In [ ]:
# Exercise 1 Solution Template
import os
import sys
import json
import platform
from datetime import datetime
from pathlib import Path

def create_system_info_tool():
    """Create a comprehensive system information tool"""
    
    system_info = {
        "timestamp": datetime.now().isoformat(),
        "system": {
            "platform": platform.system(),
            "platform_version": platform.version(),
            "architecture": platform.architecture()[0],
            "processor": platform.processor() or "Unknown"
        },
        "python": {
            "version": sys.version,
            "executable": sys.executable,
            "path": sys.path[:3]  # First 3 entries to avoid clutter
        },
        "directories": {
            "current_working_directory": os.getcwd(),
            "home_directory": os.path.expanduser("~"),
            "temp_directory": os.path.dirname(tempfile.gettempdir())
        },
        "environment": {
            "PATH": os.environ.get("PATH", "Not found")[:100] + "...",  # Truncated
            "PYTHONPATH": os.environ.get("PYTHONPATH", "Not set"),
            "USER": os.environ.get("USER") or os.environ.get("USERNAME", "Unknown")
        },
        "current_directory_contents": []
    }
    
    # TODO: Complete this function
    # Add code to:
    # 1. List files in current directory with sizes
    # 2. Calculate total directory size
    # 3. Save to JSON file
    
    print("📊 System Information Tool")
    print("=" * 40)
    print("🖥️  System:", system_info["system"]["platform"])
    print("🐍 Python:", sys.version.split()[0])
    print("📁 Working Dir:", system_info["directories"]["current_working_directory"])
    
    return system_info

# Run the tool
info = create_system_info_tool()

# Your task: Complete the implementation!

## Exercise 2: Log File Analyzer 📋

Create a program that analyzes log files and generates reports:

**Requirements:**
- Read log files with different severity levels
- Count occurrences of each log level
- Find the most common error messages
- Generate a summary report
- Save results in both JSON and CSV formats

**Sample log format:**
```
2025-08-19 10:30:15 | INFO     | User login successful: alice
2025-08-19 10:31:22 | WARNING  | Slow query detected: SELECT * FROM users
2025-08-19 10:32:45 | ERROR    | Database connection failed
```

## Exercise 3: Backup Utility 💾

Build a simple backup utility:

**Requirements:**
- Create compressed archives of specified directories
- Support both ZIP and TAR formats
- Include timestamp in backup filename
- Log backup operations
- Verify backup integrity after creation
- Clean up old backups (keep only last N backups)

## Exercise 4: Configuration Manager ⚙️

Create a configuration management system:

**Requirements:**
- Read configuration from INI files
- Provide default values for missing settings
- Validate configuration values
- Support environment variable overrides
- Save modified configurations back to file
- Support both development and production configs

## Challenge: Process Monitor 🔍

**Advanced Challenge:** Create a process monitoring tool that:

1. Lists running processes (use `subprocess` with system commands)
2. Monitors CPU and memory usage
3. Logs process information periodically
4. Sends alerts when processes exceed thresholds
5. Stores historical data in JSON format
6. Generates summary reports

**Bonus Features:**
- Web dashboard using simple HTTP server
- Email notifications for critical alerts
- Configuration via INI files
- Automatic log rotation

# 📚 Summary and Best Practices

## Key Takeaways

### 🏗️ **System Operations**
- **Always use try-except blocks** when working with file systems
- **Prefer `pathlib.Path`** over `os.path` for modern Python (3.4+)
- **Use context managers** (`with` statements) for file operations
- **Check for cross-platform compatibility** when using system commands

### 🔧 **Process Management**
- **Use `subprocess.run()`** for simple command execution
- **Always set timeouts** to prevent hanging processes
- **Capture both stdout and stderr** for proper error handling
- **Use `text=True`** to work with strings instead of bytes

### 📝 **Logging Best Practices**
- **Configure logging early** in your application
- **Use appropriate log levels**: DEBUG < INFO < WARNING < ERROR < CRITICAL
- **Include meaningful context** in log messages
- **Use structured logging** with consistent formatting
- **Consider log rotation** for production applications

### 💾 **Data Serialization Guidelines**

| Format | Use When | Pros | Cons |
|--------|----------|------|------|
| **JSON** | Web APIs, config files, cross-language | Human readable, widely supported | Limited data types |
| **Pickle** | Python-only, complex objects | Preserves Python types, fast | Not human readable, Python-specific |
| **CSV** | Tabular data, Excel compatibility | Simple, widely supported | Limited structure |
| **INI** | Configuration files | Human readable, simple | Limited nesting |

### 🗜️ **Compression and Archives**
- **Use ZIP** for cross-platform compatibility
- **Use TAR.GZ** for better compression on Unix systems
- **Always verify** archive integrity after creation
- **Consider compression level** vs. speed trade-offs

## 🚨 Common Pitfalls to Avoid

1. **Not handling exceptions** when working with files/processes
2. **Forgetting to close resources** (use `with` statements!)
3. **Hard-coding file paths** (use `os.path.join()` or `pathlib`)
4. **Not validating user input** before passing to system commands
5. **Ignoring return codes** from subprocess operations
6. **Not setting timeouts** for external processes
7. **Using deprecated functions** (`os.system()` instead of `subprocess`)
8. **Not considering security** when executing external commands

## 🎯 Production Readiness Checklist

- [ ] **Error handling**: All operations wrapped in try-except
- [ ] **Logging**: Comprehensive logging with appropriate levels
- [ ] **Configuration**: Externalized configuration files
- [ ] **Security**: Input validation and safe command execution
- [ ] **Performance**: Timeouts and resource management
- [ ] **Monitoring**: Process monitoring and health checks
- [ ] **Documentation**: Clear docstrings and comments
- [ ] **Testing**: Unit tests for critical functionality

## 🔗 Further Learning Resources

- **Official Documentation**: [Python Standard Library](https://docs.python.org/3/library/)
- **PEP 8**: Python Style Guide
- **Real Python**: Tutorials on system programming
- **Automate the Boring Stuff**: Practical Python programming
- **Effective Python**: Advanced best practices

---

## 🎉 Congratulations!

You've completed the Python Built-in Functions and Runtime Library guide! You now have the knowledge to:

- ✅ Interact with the operating system programmatically
- ✅ Manage files, directories, and processes
- ✅ Implement robust logging and error handling
- ✅ Work with various data formats and compression
- ✅ Build production-ready Python applications

**Next Steps:** Try the exercises above and start building your own system utilities!